## Setup

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop
from kepler.pandas.decorators import RingerDecorator
from typing import List
from itertools import product
import rootplotlib as rpl
import mplhep as hep
import ROOT
ROOT.gStyle.SetOptStat(0);
import numpy as np
import pandas as pd
import glob
import os
import logging
import matplotlib.pyplot as plt
plt.style.use(hep.style.ROOT)
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from copy import deepcopy
from argparse import ArgumentParser
import json
from Gaugi import GeV
from datetime import datetime

from packages.generators import ring_percentages, RingGenerator
from packages.plotting import make_plot_fig, var_infos, val_label_map
from packages.utils import get_logger

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


In [2]:
drop_cols = drop_columns = [
                    'RunNumber', 
                    'trig_L2_cl_e2tsts1',
                    'trig_L2_el_hastrack',
                    'trig_L2_el_eta',
                    'trig_L2_el_phi',
                    'trig_L2_el_caloEta',
                    'trig_L2_el_trkClusDeta',
                    'trig_L2_el_trkClusDphi',
                    'trig_L2_el_etOverPt',
                    'trig_EF_cl_hascluster',
                    'trig_EF_cl_eta',
                    'trig_EF_cl_etaBE2',
                    'trig_EF_cl_phi',     
                    'trig_EF_el_hascand',
                    'trig_EF_el_eta',
                    'trig_EF_el_etaBE2',
                    'trig_EF_el_phi',
                    'trig_EF_el_rhad1',
                    'trig_EF_el_rhad',
                    'trig_EF_el_f3',
                    'trig_EF_el_weta2',
                    'trig_EF_el_rphi',
                    'trig_EF_el_reta',
                    'trig_EF_el_wtots1',
                    'trig_EF_el_eratio',
                    'trig_EF_el_f1',
                    'trig_EF_el_hastrack',
                    'trig_EF_el_deltaEta1',
                    'trig_EF_el_deltaPhi2',
                    'trig_EF_el_deltaPhi2Rescaled',
                    'el_etaBE2',
                    'el_numberOfBLayerHits',
                    'el_numberOfPixelHits',
                    'el_numberOfTRTHits',
                    'el_trans_TRT_PID',
                    'el_deltaPhi2',
                    'el_TaP_Mass',
                ]

l1seeds_per_energy = {
    24: 'L1_EM22VHI',
    26: 'L1_EM22VHI',
    60: 'L1_EM24VHI',
    140: 'L1_EM24VHI'
}

criteria_conf_names = {
    'tight': 'ElectronRingerTightTriggerConfig.conf',
    'medium': 'ElectronRingerMediumTriggerConfig.conf',
    'loose': 'ElectronRingerLooseTriggerConfig.conf',
    'vloose': 'ElectronRingerVeryLooseTriggerConfig.conf'
}

energy_chains = {
     24: 'e24_lhtight_nod0_{strategy}_ivarloose',
     26: 'e26_lhtight_nod0_{strategy}_ivarloose',
     60: 'e60_lhmedium_nod0_{strategy}_L1EM24VHI',
     140: 'e140_lhloose_nod0_{strategy}'
}

trigger_steps = ['L2Calo', 'L2', 'EFCalo', 'HLT']

et_bins = [15, 20, 30, 40, 50, 1000000]
et_bins_idxs = range(len(et_bins)-1)
eta_bins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]
eta_bins_idxs = range(len(eta_bins)-1)

## Parameters

In [3]:
HOME_PATH = os.path.expanduser('~')
datasetpath = os.path.join(HOME_PATH, 
    '/data/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2')

modelpaths = [
    'tunings/TrigL2_20180125_v8'
    # 'tunings/TrigL2_20210907_v12 (v8.1-2)',
    # 'tunings/TrigL2_20220704_v20',
    # 'tunings/TrigL2_20220802_v20.1'
    # 'tunings/TrigL2_20210306_v9'
]

cutbased = False

chain_names = [
    'e24_lhtight_nod0_{strategy}_ivarloose',
    'e26_lhtight_nod0_{strategy}_ivarloose',
    'e60_lhmedium_nod0_{strategy}_L1EM24VHI',
    'e140_lhloose_nod0_{strategy}'
]

dev = False
log = False

# Initializing logger
simulation_logger = get_logger('simulate_chains', file=log)

## Building Decorators

In [4]:
simulation_logger.info('Building decorators')
decorators = list()
trigger_strategies = ['noringer'] if cutbased else list()
strategy_cols = defaultdict(list)
for modelpath, criterion in product(modelpaths, criteria_conf_names.keys()):
    conf_name = criteria_conf_names[criterion]
    confpath = os.path.join(modelpath, conf_name)
    env = ROOT.TEnv(confpath)
    ringer_version = env.GetValue("__version__", '')
    ringer_name = f'ringer_{ringer_version}'
    strat_criterion = f'{ringer_name}_{criterion}'
    simulation_logger.info(f'Building decorator for {confpath}. Version: {ringer_version}')
    decorator = RingerDecorator(strat_criterion, confpath, RingGenerator(ring_percentages[ringer_version]))
    decorators.append(decorator)
    strategy_cols[ringer_name].append(strat_criterion)
    strategy_cols[ringer_name].append(strat_criterion + '_output')
    if not ringer_name in trigger_strategies:
        trigger_strategies.append(ringer_name)

2022-08-30 09:26:47,162 - Building decorators
2022-08-30 09:26:47,175 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerTightTriggerConfig.conf. Version: v8
2022-08-30 09:26:47,676 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerMediumTriggerConfig.conf. Version: v8
2022-08-30 09:26:48,106 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.conf. Version: v8
2022-08-30 09:26:48,500 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerVeryLooseTriggerConfig.conf. Version: v8


2022-08-30 09:26:47,177 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerTightTriggerConfig.conf
2022-08-30 09:26:47,678 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerMediumTriggerConfig.conf
2022-08-30 09:26:48,107 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.conf
2022-08-30 09:26:48,502 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerVeryLooseTriggerConfig.conf


2022-08-30 09:26:47.191537: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-08-30 09:26:47.191574: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: UNKNOWN ERROR (-1)
2022-08-30 09:26:47.191593: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (caloba53): /proc/driver/nvidia/version does not exist
2022-08-30 09:26:47.191840: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-30 09:26:47.225944: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2400000000 Hz
2022-08-30 09:26:47.234935: I tensorflow/compiler/xla/service/service.c

## Building Chains

In [5]:
simulation_logger.info('Building chains')
chains = list()
step_chain_names = list()
for chain_name, strategy in product(chain_names, trigger_strategies):
    spliited_chain_name = chain_name.split('_')
    criterion = spliited_chain_name[1].replace('lh', '')
    step_chain_name = f'HLT_{chain_name.format(strategy=strategy)}'
    step_chain_names.append(step_chain_name)
    strategy_cols[strategy].append(step_chain_name)
    for trigger_step in trigger_steps:
        if trigger_step != 'HLT':
            lower_chain = step_chain_name.replace('HLT', trigger_step)
            step_chain_names.append(lower_chain)
            strategy_cols[strategy].append(lower_chain)
    energy = int(spliited_chain_name[0][1:])
    l1seed = l1seeds_per_energy[energy]
    l2calo_column = f'{strategy}_{criterion}'
    simulation_logger.info(f'Building chain: {step_chain_name} model: {l2calo_column}')
    if strategy == 'noringer':
        chain = Chain(step_chain_name, L1Seed=l1seed)
    else:
        chain = Chain(step_chain_name, L1Seed=l1seed, l2calo_column=l2calo_column)
    chains.append(chain)

2022-08-30 09:26:48,933 - Building chains
2022-08-30 09:26:48,935 - Building chain: HLT_e24_lhtight_nod0_ringer_v8_ivarloose model: ringer_v8_tight
2022-08-30 09:26:48,936 - Building chain: HLT_e26_lhtight_nod0_ringer_v8_ivarloose model: ringer_v8_tight
2022-08-30 09:26:48,937 - Building chain: HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI model: ringer_v8_medium
2022-08-30 09:26:48,938 - Building chain: HLT_e140_lhloose_nod0_ringer_v8 model: ringer_v8_loose


OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v8_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (23000.0, 'ringer_v8_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 26000.0),
             ('HLT', (26000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v8_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v8_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Simulating

In [7]:
simulation_logger.info('Simulating')
filename_end = '*et4_eta0.npz' if dev else '*.npz'    #If dev, load only a part of the dataset
datafiles = glob.glob(os.path.join(datasetpath, filename_end))  
simulation_logger.info(f'glob_path: {os.path.join(datasetpath, filename_end)}')
data = load_in_loop(datafiles, drop_columns=drop_cols, decorators=decorators, chains=chains)

2022-08-30 09:26:48,969 - Simulating
2022-08-30 09:26:48,972 - glob_path: /home/lbarranunes/data/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2/*.npz
Reading files...:   4%|▍         | 1/25 [00:00<00:13,  1.77it/s]

L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


Reading files...:   8%|▊         | 2/25 [00:00<00:11,  2.09it/s]

L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose


Reading files...:  12%|█▏        | 3/25 [00:01<00:09,  2.31it/s]

...

L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


Reading files...:  96%|█████████▌| 24/25 [00:11<00:00,  1.46it/s]

L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


Reading files...: 100%|██████████| 25/25 [00:12<00:00,  2.05it/s]

L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


## Saving data

In [8]:
simulation_logger.info('Saving_data')
output_dir = os.path.join(datasetpath, 'simulated_chains')
save_cols = [f'el_lh{criterion}' for criterion in criteria_conf_names.keys()]
save_cols += ['trig_L2_el_pt', 'avgmu', 'el_et', 'el_eta', 'el_TaP_deltaR']
save_cols += ['target']
last_strat = None

2022-08-30 09:27:01,312 - Saving_data


In [9]:
for strategy, et_bin_idx, eta_bin_idx in product(strategy_cols.keys(), et_bins_idxs, eta_bins_idxs):
    strategy = 'cutbased' if strategy == 'noringer' else strategy
    outname = f'{strategy}_et{et_bin_idx}_eta{eta_bin_idx}'
    simulation_logger.info(f'Generating {outname}')
    strategy_out = os.path.join(output_dir, strategy + '.parquet')
    if not os.path.exists(strategy_out):
        os.makedirs(strategy_out)

    selection_cols = strategy_cols[strategy] + save_cols
    if last_strat != strategy:
        with open(os.path.join(output_dir, f'{strategy}_cols.json'), 'w') as json_file:
            json.dump(selection_cols, json_file, indent=4)
        last_strat = deepcopy(strategy)

    et_min, et_max = et_bins[et_bin_idx:et_bin_idx+2]
    eta_min, eta_max = eta_bins[eta_bin_idx:eta_bin_idx+2]
    bin_selector = (data['el_et'] >= (et_min*GeV)) & (data['el_et'] < (et_max*GeV))
    bin_selector = bin_selector & (data['el_eta'].abs() >= eta_min) & (data['el_eta'].abs() < eta_max)
    selected_data = data.loc[bin_selector, selection_cols]

    simulation_logger.info(f'Saving: {outname}')
    selected_data.to_parquet(os.path.join(strategy_out, outname + '.parquet'))

2022-08-30 09:27:01,334 - Generating ringer_v8_et0_eta0
2022-08-30 09:27:01,382 - Saving: ringer_v8_et0_eta0
2022-08-30 09:27:01,487 - Generating ringer_v8_et0_eta1
2022-08-30 09:27:01,492 - Saving: ringer_v8_et0_eta1
2022-08-30 09:27:01,535 - Generating ringer_v8_et0_eta2
2022-08-30 09:27:01,539 - Saving: ringer_v8_et0_eta2
2022-08-30 09:27:01,580 - Generating ringer_v8_et0_eta3
2022-08-30 09:27:01,583 - Saving: ringer_v8_et0_eta3
2022-08-30 09:27:01,624 - Generating ringer_v8_et0_eta4
2022-08-30 09:27:01,627 - Saving: ringer_v8_et0_eta4
2022-08-30 09:27:01,667 - Generating ringer_v8_et1_eta0
2022-08-30 09:27:01,671 - Saving: ringer_v8_et1_eta0
2022-08-30 09:27:01,715 - Generating ringer_v8_et1_eta1
2022-08-30 09:27:01,719 - Saving: ringer_v8_et1_eta1
2022-08-30 09:27:01,762 - Generating ringer_v8_et1_eta2
2022-08-30 09:27:01,766 - Saving: ringer_v8_et1_eta2
2022-08-30 09:27:01,809 - Generating ringer_v8_et1_eta3
2022-08-30 09:27:01,813 - Saving: ringer_v8_et1_eta3
2022-08-30 09:27:01

## Loading

In [13]:
start = datetime.now()
read_data = pd.read_parquet(os.path.join(HOME_PATH, 
    '/data/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2/simulated_chains/ringer_v8.parquet'))
end = datetime.now()
print(f'Read time: {end-start}')

Read time: 0:00:00.115821


In [12]:
for col in read_data.columns:
    if not col.startswith('trig_L2_cl_ring'):
        print(col)

ringer_v8_tight
ringer_v8_tight_output
ringer_v8_medium
ringer_v8_medium_output
ringer_v8_loose
ringer_v8_loose_output
ringer_v8_vloose
ringer_v8_vloose_output
HLT_e24_lhtight_nod0_ringer_v8_ivarloose
L2Calo_e24_lhtight_nod0_ringer_v8_ivarloose
L2_e24_lhtight_nod0_ringer_v8_ivarloose
EFCalo_e24_lhtight_nod0_ringer_v8_ivarloose
HLT_e26_lhtight_nod0_ringer_v8_ivarloose
L2Calo_e26_lhtight_nod0_ringer_v8_ivarloose
L2_e26_lhtight_nod0_ringer_v8_ivarloose
EFCalo_e26_lhtight_nod0_ringer_v8_ivarloose
HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
L2Calo_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
L2_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
EFCalo_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
HLT_e140_lhloose_nod0_ringer_v8
L2Calo_e140_lhloose_nod0_ringer_v8
L2_e140_lhloose_nod0_ringer_v8
EFCalo_e140_lhloose_nod0_ringer_v8
el_lhtight
el_lhmedium
el_lhloose
el_lhvloose
trig_L2_el_pt
avgmu
el_et
el_eta
el_TaP_deltaR
target
